In [1]:

!pip install uvicorn
!pip install fastapi
!pip install pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of

In [17]:
!pip install geopandas

In [20]:
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y
!pip install rioxarray
!pip install rasterio

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [47.2 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,520 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,284 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,498 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,015 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,467 kB]
Get:13 http://security.ubu

In [25]:
ls

foss4g/  sample_data/


In [26]:
cd foss4g

/content/foss4g


In [36]:
# put the parameter in your request
from fastapi import FastAPI, Query
import geojson
import shapely.wkt
import json
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np
from rioxarray import open_rasterio
import ast
import os
import json
from osgeo import gdal

@app.get('/_area')
async def root(coords: str = Query(None, description="A test parameter")):

  wkt_data = shapely.wkt.loads(coords)
  wkt_data2 = geojson.Feature(geometry=wkt_data, properties={})

  dump_json = json.dumps(wkt_data2)
  gdf = gpd.read_file(dump_json)

  # Open the raster file to be clipped
  raster_file = 'ndvi.tif'
  src = rasterio.open(raster_file)
  print(src)

  # Clip the raster using the GeoJSON geometry
  clipped_data, clipped_transform = mask(src, gdf.geometry, crop=True)

  clipped_meta = src.meta
  clipped_meta.update({
      'height': clipped_data.shape[1],
      'width': clipped_data.shape[2],
      'transform': clipped_transform
  })

  # Define the output file name
  output_file = 'clipped_raster.tif'

  # Create a new raster file with the clipped data and updated metadata
  with rasterio.open(output_file, 'w', **clipped_meta) as dst:
      dst.write(clipped_data)

  print(f"Clipped raster saved to {output_file}")


  # Open the clipped raster file
  output_file = 'clipped_raster.tif'
  with rasterio.open(output_file) as ds_nir_org:
      # Read the pixel values
    data = ds_nir_org.read(1)

    # Specify the background value (replace 0 with your background value)
    background_value = 0.0

    # Mask the background values
    masked_data = np.ma.masked_equal(data, background_value)

    # Convert the masked data to a list
    text = masked_data.tolist()

    text2 = str(text).replace('[', '')
    text3 = str(text2).replace(']', '')
    text4 = str(text3).replace('Nan', str(None))


    arr = ast.literal_eval(text4)

    my_list = list(arr)


    dateImage = ['2010-01-01T00:12:20Z']
    crt = len(dateImage)

    # Opening JSON file
    f = open('coverage.json')

    # see detail into file coverage.json

    covJSON = json.load(f)

    covJSON['ranges']['data']['values'] = my_list

    get_file = output_file
    dataset = gdal.Open(get_file)
    cols = dataset.RasterXSize
    rows = dataset.RasterYSize


    def GetExtent(ds):
        xmin, xpixel, _, ymax, _, ypixel = ds.GetGeoTransform()
        width, height = ds.RasterXSize, ds.RasterYSize
        xmax = xmin + width * xpixel
        ymin = ymax + height * ypixel

        return [xmin, ymin, xmax, ymax]
    ds = dataset
    extent = GetExtent(ds)

    axes = {
        "x": {
            "start": extent[0],
            "stop": extent[2],
            "num": cols
        },
        "y": {
            "start": extent[3],
            "stop": extent[1],
            "num": rows
        },
        "t": {"values": dateImage}
    }

    covJSON['domain']['axes'] = axes

    shape = [crt, rows, cols]

    covJSON['ranges']['data']['shape'] = shape

    msgRetren = covJSON


  return msgRetren

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://ad48-34-125-106-19.ngrok.io


INFO:     Started server process [189]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


<open DatasetReader name='ndvi.tif' mode='r'>
Clipped raster saved to clipped_raster.tif
INFO:     175.192.226.112:0 - "GET /_area?coords=POLYGON%20((100.40375586163655%2014.853583642788516,%20100.40492627187803%2014.851100089349307,%20100.40795221055109%2014.852584512094582,%20100.40655342757957%2014.855039518942535,%20100.40375586163655%2014.853583642788516)) HTTP/1.1" 200 OK
INFO:     175.192.226.112:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
<open DatasetReader name='ndvi.tif' mode='r'>
Clipped raster saved to clipped_raster.tif
INFO:     175.192.226.112:0 - "GET /_area?coords=POLYGON%20((100.40375586163655%2014.853583642788516,%20100.40492627187803%2014.851100089349307,%20100.40795221055109%2014.852584512094582,%20100.40655342757957%2014.855039518942535,%20100.40375586163655%2014.853583642788516)) HTTP/1.1" 200 OK
